In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from IPython.display import display
pd.set_option('max_rows', 5)
%matplotlib inline

## 過学習 (overfitting)
---
訓練データでは高い精度が出るのに、未知のデータ (新しく入ってきたデータ) ではそれほど精度が出ないことがよくある。この状態を過学習や過剰適合といい、**モデルが訓練データに対して過剰に適合**した結果である。

## 汎化 (generalization)
---
未知のデータでも訓練データと同じような精度が出るモデルは汎化能力が高いという。  
訓練データでも未知のデータでも求める精度が出ない状態のことは未学習 (underfitting) という。  
モデルのパラメーター (線形回帰の係数のように訓練データから自動的に調整される変数) が多く過学習しやすいモデルのことを高バリアンス (high variance) 、逆にパラメータが少なく未学習が起きやすいモデルのことを高バイアス (high bias) であるという。 

<table class="border text-center background-bright">
    <tr class="background-dark">
        <th class="border-right-bold"></th>
        <th>訓練データで高精度</th>
        <th>訓練データで低精度</th>
    </tr>
    <tr>
        <th class="border-bottom border-right-bold background-dark">未知のデータで高精度</th>
        <td>理想</td>
        <td>-</td>
    </tr>
    <tr>
        <th class="border-bottom border-right-bold background-dark">未知のデータで低精度</th>
        <td>過学習・高バリアンス</td>
        <td>未学習・高バイアス</td>
    </tr>
</table>

過学習は、**訓練データに対してモデルが複雑すぎる**ために起きる。

<table class="text-center border">
    <tr class="background-dark">
        <th>データとモデルの複雑さの関係</th>
        <th>現象</th>
    </tr>
    <tr>
        <td>データの複雑さ &lt; モデルの複雑さ (モデルの学習能力)</td>
        <td>過学習</td>
    </tr>
    <tr>
        <td>データの複雑さ &gt; モデルの複雑さ (モデルの学習能力)</td>
        <td>未学習</td>
    </tr>
</table>

以下のように、モデルの能力が低い (高バイアスなモデル) とデータにうまくフィットしないが、モデルの能力が高すぎる (高バリアンスなモデル) と学習データには完璧にフィットするが真の分布から新しいデータを得た場合にうまく予測できない。

In [2]:
from helpers.test import generalization
generalization.show()

interactive(children=(IntSlider(value=8, continuous_update=False, description='モデルの複雑さ', max=20, min=1, style=…

## 評価・テスト (test)
---
未知のデータに学習させたモデルを適用して汎化性能を測定すること。  
機械学習の目的は、未知のデータに対する予測なので、未知のデータで精度が出るかどうかが重要であり、それを測定して確認する必要がある。

ただし、モデルはあくまで実際の現象を単純化したものであり、テストを実施しているからといって過度に一般化するのは避ける。  
例えば、**データセットに含まれない範囲の値を入力する外挿は避けるべき**であると言われる。モデルが複雑な関数の一部分を切り出して単純な関数に近似 (3次関数の一部を1次関数で近似など) している可能性もあるからである。

また、**テストの主な目的は過学習の確認** (訓練データとテストデータでの精度の差の確認) であり、テストの結果 (精度) を信頼しすぎるのも注意が必要。実際に運用してみるとテストよりも精度が悪いことはよくある。

### テストの方法
---
手元にないデータが集まるまでモデルを評価できないのでは都合が悪いので、通常は**手元にあるデータを訓練データとテストデータに分け**、訓練データでモデルのパラメーターを決定しテストデータでモデルを評価する。

<table class="border text-center">
    <tr class="background-dark">
        <td colspan="2">データセット</td>
    </tr>
    <tr class="background-dark">
        <td>訓練データ</td>
        <td>テストデータ</td>
    </tr>
    <tr class="background-bright">
        <td>モデルのパラメーターを決定</td>
        <td>モデルの汎化性能を測定</td>
    </tr>
</table>

一貫した評価のため、テストデータには評価する対象のモデル間 (使用する変数が少ないモデルと多いモデルなど) で**毎回同じデータセットを用いる**。また、テストデータに偏りがあると正しい評価ができないため、**慎重に設計**する必要がある。
- 正解ラベルに偏りがないか
- 特徴の分布に偏りがないか
- etc

訓練データとテストデータの割合は6:4、7:3、8:2のいずれかで分割して使用するのが一般的だが、データセットが十分に大きい場合は、9:1や99:1も用いられる。

### Pythonでのデータセットの分割方法
---
`sklearn.model_selection.train_test_split`を使用する。データに偏りがないようにシャッフルされるので、再現性を確保するために**`random_state`は必ず指定**する。

In [3]:
n_features = 5


def to_dataframe(x, y):
    return pd.DataFrame(np.column_stack([x, y]),
                        columns=[f'$x_{i+1}$' for i in range(x.shape[1])] +
                        ['$y$']).reset_index()


x_reg, y_reg = make_regression(n_features=5, noise=0.1, random_state=1234)
reg = to_dataframe(x_reg, y_reg)
print('reg')
display(reg)

x_clf, y_clf = make_classification(n_features=n_features, random_state=1234)
clf = to_dataframe(x_clf, y_clf)
print('clf')
display(clf)

reg


,index,$x_1$,$x_2$,$x_3$,$x_4$,$x_5$,$y$
0,0,-0.226632,-1.270063,-0.923831,0.355839,-0.195472,-88.673382
1,1,-0.041387,1.282933,0.821048,2.097801,0.270338,151.441869
...,...,...,...,...,...,...,...
98,98,-1.154084,-0.413279,-1.380173,-0.495833,-1.719317,-332.315818
99,99,0.725714,-1.522180,0.916976,-0.563890,-0.014279,100.990582


clf


,index,$x_1$,$x_2$,$x_3$,$x_4$,$x_5$,$y$
0,0,-0.064810,0.942149,0.693305,0.594743,0.171602,1.0
1,1,0.252833,-0.908485,-0.879626,-0.940067,0.500483,0.0
...,...,...,...,...,...,...,...
98,98,-0.059355,-1.216396,-0.736489,-0.492400,-0.276646,0.0
99,99,-0.109198,0.996937,0.778670,0.707556,-0.319298,1.0


In [4]:
train_test_split??

In [5]:
x_reg = reg.iloc[:, :-1].values
y_reg = reg.iloc[:, -1].values
x_reg_train, x_reg_test, y_reg_train, y_reg_test = train_test_split(
    x_reg, y_reg, test_size=0.1, random_state=1234)

# 結果の確認
reg_train = pd.DataFrame(np.column_stack([x_reg_train, y_reg_train]),
                         columns=reg.columns)
reg_train['index'] = reg_train['index'].astype(int)
reg_test = pd.DataFrame(np.column_stack([x_reg_test, y_reg_test]),
                        columns=reg.columns)
reg_test['index'] = reg_test['index'].astype(int)
print('reg_train')
display(reg_train)
print('reg_test')
display(reg_test)

reg_train


,index,$x_1$,$x_2$,$x_3$,$x_4$,$x_5$,$y$
0,33,0.513082,2.107785,-0.546416,1.043945,1.459927,147.415936
1,59,-0.430096,0.270836,0.767369,0.984920,1.391986,151.080687
...,...,...,...,...,...,...,...
88,83,1.015405,0.440266,0.749185,-0.675521,0.688972,162.855411
89,47,-0.897157,0.755414,-0.136795,0.018289,0.215269,-66.752940


reg_test


,index,$x_1$,$x_2$,$x_3$,$x_4$,$x_5$,$y$
0,40,1.343182,-1.553342,-0.050540,-0.364010,-0.319298,69.510039
1,35,0.264534,-0.174298,0.565658,0.585084,-1.071369,8.398147
...,...,...,...,...,...,...,...
8,39,0.209395,-0.896581,-0.592886,-1.473116,1.104352,-2.714533
9,42,-0.048461,0.425531,-0.845290,0.415101,-0.980724,-116.393952


In [6]:
x_clf = clf.iloc[:, :-1].values
y_clf = clf.iloc[:, -1].values
x_clf_train, x_clf_test, y_clf_train, y_clf_test = train_test_split(
    x_clf, y_clf, test_size=0.1, random_state=1234)

# 結果の確認
clf_train = pd.DataFrame(np.column_stack([x_clf_train, y_clf_train]),
                         columns=clf.columns)
clf_train['index'] = clf_train['index'].astype(int)
clf_test = pd.DataFrame(np.column_stack([x_clf_test, y_clf_test]),
                        columns=clf.columns)
clf_test['index'] = clf_test['index'].astype(int)
print('clf_train')
display(clf_train)
print('clf_test')
display(clf_test)

clf_train


,index,$x_1$,$x_2$,$x_3$,$x_4$,$x_5$,$y$
0,33,-0.762661,-1.242861,0.026055,0.848905,-1.051628,1.0
1,59,0.685935,-0.319168,-0.971251,-1.480243,0.440266,0.0
...,...,...,...,...,...,...,...
88,83,-0.948686,-0.893450,0.462831,1.381451,-1.268994,1.0
89,47,-1.250956,-1.747812,0.234538,1.537458,1.076541,1.0


clf_test


,index,$x_1$,$x_2$,$x_3$,$x_4$,$x_5$,$y$
0,40,0.439331,0.839825,0.066697,-0.427226,-0.660230,0.0
1,35,-0.397057,0.581005,0.823577,1.054490,-0.061246,1.0
...,...,...,...,...,...,...,...
8,39,0.380907,-1.851514,-1.643674,-1.657088,0.688972,0.0
9,42,0.983739,0.034863,-1.068016,-1.877203,0.226363,0.0


### Pythonでのテスト方法
---
学習済みモデルの`score`メソッドを使用する。  
評価指標は回帰モデルでは決定係数 ($R^{2}$) 、分類モデルでは正解率 (accuracy) が使用される。指標の詳細・その他の指標については[評価指標](metrics.ipynb)で扱う。

In [7]:
# 線形回帰
model_reg = LinearRegression()
model_reg.fit(x_reg_train, y_reg_train)
model_reg.score(x_reg_test, y_reg_test)

0.9999993658171609

In [8]:
# ロジスティック回帰
model_clf = LogisticRegression()
model_clf.fit(x_clf_train, y_clf_train)
model_clf.score(x_clf_test, y_clf_test)

/root/local/python-3.7.4/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9